In [112]:
from sys import getsizeof

dataset_name = "covid-sequence.txt"

f = open(dataset_name, "r")
lines = f.readlines()
genome = ''.join(''.join(lines).split())
genome

def write_to_file(filename, str_compress, bytefile=False):
    if bytefile:
        file1 = open(filename,"wb")
    else:
        file1 = open(filename,"w")
    file1.write(str_compress)
    file1.close()

def compare_file(filename1, filename2):
    import os
    size_a = os.path.getsize(filename1)
    size_b = os.path.getsize(filename2)
    print("Size of fileA: ", size_a)
    print("Size of fileB: ", size_b)

In [113]:
# Basic Run length encoding 
MAX_RUN = 4 #not being used currently
genome_size = len(genome)

def run_length_coding(sequence):
    compressed = ""
    size = len(sequence)
    i = 0
    while i < size: 
        run_length = 1
        k = i+1
        while k < size and sequence[i]==sequence[k]:
            run_length += 1
            k += 1
    
        compressed += sequence[i] + str(run_length)*( 0 if run_length==1 else 1)
        i = k   
    return compressed

result = run_length_coding(genome)
filename = "algo-1.txt"
write_to_file(filename,result)
compare_file(filename, dataset_name)

Size of fileA:  27727
Size of fileB:  30330


In [114]:
# Gzip compression
import gzip
compressed_gzip = gzip.compress(bytes(genome, 'utf-8'))

# print (compressed_gzip)
# a = getsizeof(compressed_gzip)
# b = getsizeof(genome)
# c = getsizeof( bytes('1067789', 'utf-8') )
# print (a,b, c)
filename = "algo-gzip.bin"
write_to_file(filename, compressed_gzip, True)
compare_file(filename, dataset_name)

Size of fileA:  8953
Size of fileB:  30330


In [115]:
# Bzip2 compression
import bz2
compressed_bzip = bz2.compress(bytes(genome, 'utf-8'))
print (len(compressed_bzip))
filename = "algo-bzip.bin"
write_to_file(filename, compressed_bzip, True)
compare_file(filename, dataset_name)

8158
Size of fileA:  8158
Size of fileB:  30330


In [119]:
# Split encoding
a_rle = []
g_rle = []
c_rle = []
t_rle = []
for i in range(genome_size):
    if genome[i] == 'a':
        a_rle.append('1')
    else:
        a_rle.append('0')

    if genome[i] == 'g':
        g_rle.append('1')
    else:
        g_rle.append('0')

    if genome[i] == 'c':
        c_rle.append('1')
    else:
        c_rle.append('0')

    if genome[i] == 't':
        t_rle.append('1')
    else:
        t_rle.append('0')
print (a_rle[164:180], genome[130:180])
print (c_rle[164:180])

['0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0'] GAGTAACTCGTCTATC
['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1']


In [121]:
def split_encoding(sequence):
    sequence = ['0'] + sequence if sequence[0]=='1' else sequence #start every sequence with 0
    compressed = []
    size = len(sequence)
    i = 0
    while i < size: 
        run_length = 1
        k = i+1
        while k < size and sequence[i]==sequence[k]:
            run_length += 1
            k += 1
        compressed += [run_length]
        i = k
    # new_compress = ''
    # print (compressed[164:189], compressed[189])
    # for i in range(len(compressed)):
    #     if i%2==1:
    #         new_compress += compressed[i] + f"-{i} : "
    return compressed

a_bin = split_encoding(a_rle)
g_bin = split_encoding(g_rle)
c_bin = split_encoding(c_rle)
t_bin = split_encoding(t_rle)

print (a_bin[0:10])
print (a_rle[0:20])
# print (split_encoding(g_rle))

[1, 1, 2, 3, 5, 1, 1, 1, 7, 1]
['1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0']


In [134]:
# Traverses tree to assign bit strings to leaf nodes
def generate_bits(symbol_to_bits, n, bits = ''):
    if n.is_leaf():
        symbol_to_bits[n.s] = bits
        return

    generate_bits(symbol_to_bits, n.left, bits = bits + '1')
    generate_bits(symbol_to_bits, n.right, bits = bits + '0')

a_symbol_bits = {}
generate_bits(a_symbol_bits, a_root_node)
t_symbol_bits = {}
generate_bits(t_symbol_bits, t_root_node)
g_symbol_bits = {}
generate_bits(g_symbol_bits, g_root_node)
c_symbol_bits = {}
generate_bits(c_symbol_bits, c_root_node)

In [139]:
def assign_bits(bin, symbol_bits):
    return "".join([symbol_bits[str(sym)] for sym in bin])

compressed_a = assign_bits(a_bin[1:], a_symbol_bits)
compressed_t = assign_bits(t_bin[1:], t_symbol_bits)
compressed_g = assign_bits(g_bin[1:], g_symbol_bits)
compressed_c = assign_bits(c_bin[1:], c_symbol_bits)
print(len(compressed_a)+len(compressed_t)+len(compressed_g)+len(compressed_c))

108248


In [141]:
108248/len(genome)

3.6199712403437783

29903

In [142]:
write_to_file('covid_lower.seq',genome.lower())